In [ ]:
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import ipywidgets as widgets
import json
from IPython.display import HTML

In [ ]:
separateur = ";"
PAIRING_FILE = "./pairings.csv"

In [ ]:
df = pd.read_csv(PAIRING_FILE, sep=separateur, index_col=0)
df

In [ ]:
def has_duplicates(listOfElements):
    if len(listOfElements) == len(set(listOfElements)):
        return False
    else:
        return True
    
def sum_score_of_pairing(pairing):
    score = 0
    for (x, y) in pairing:
        score += df[x][y]
    return score

def launch_pairing(df, paired_estimates=0):
    team1 = df.columns.values.tolist()
    team2 = df.index.values.tolist()    
    if len(team1) == len(team2): 
        result = list(map(lambda team: list(itertools.product([team], team2)), team1))
        result_product = list(itertools.product(*result))
        pairings = list(filter(lambda pairing: not has_duplicates([x[1] for x in pairing]), result_product))
        df_pairings = pd.DataFrame(pd.Series(pairings), columns=["Pairings"])
        df_pairings["Estimation"] = df_pairings["Pairings"].apply(lambda x: sum_score_of_pairing(x) + paired_estimates)
        return df_pairings
    return None

def result_of(df):
    win = len(df[df.Estimation > 65].index)
    draw = len(df[(df.Estimation >= 55) & (df.Estimation <= 65)].index)
    loose = len(df[df.Estimation < 55].index)
    return [win, draw, loose]

def top_ten(df):
    return df.sort_values(by="Estimation", ascending=False).head(10)

def repartition(df):
    df = df.Estimation.value_counts().to_frame('count').reset_index().sort_values(by="index")
    df["VP"] = df["index"]
    df["Nombre"] = df["count"]
    return df[["VP", "Nombre"]].sort_values(by="VP", ascending=False)

def print_result(df, paired_estimates=0):
    result = launch_pairing(df, paired_estimates)
    results = result_of(result)
    print("Win / Draw / Loose : " + str(results[0]) + " / " + str(results[1]) + " / " + str(results[2]))
    display(top_ten(result)) 
    display(repartition(result))
    return result

def print_with_vizu(df, paired_estimates=0):
    result = launch_pairing(df, paired_estimates)
    results = result_of(result)
    print("Win / Draw / Loose : " + str(results[0]) + " / " + str(results[1]) + " / " + str(results[2]))
    plt.pie(results, labels=["Wins", "Draws", "Looses"], autopct='%1.1f%%')
    plt.show()
    plt.hist(result.Estimation)
    plt.title("Répartition des estimations")
    plt.xlabel("Victory Points")
    plt.ylabel("Nombre de pairings")
    plt.grid()
    display(top_ten(result))
    
def find_pairing(df, estimation):
    result = launch_pairing(df)
    return HTML(result[result.Estimation == estimation].to_html())

def cut_pairing(df, column, index):
    score_droped = df.iloc[index][column]
    print(df.columns[column] + " vs " + df.index[index] + " - estimé " + str(score_droped))
    new_pairing = df.drop(columns=df.columns[column]).drop(index=df.index[index])
    return new_pairing, score_droped

In [ ]:
print("#### AVANT LE PREMIER PAIRING ####\n")
print_with_vizu(df)

In [ ]:
display(df)
find_pairing(df, 79)

In [ ]:
print("#### 1ER PAIRING ####\n")
new_pairing, score_droped = cut_pairing(df, 2, 5)
#print_with_vizu(new_pairing, score_droped)

In [ ]:
print("#### 2EME PAIRING ####\n")
second_pairing, second_score_droped = cut_pairing(new_pairing, 3, 1)
#print_with_vizu(second_pairing, score_droped + second_score_droped)